In [74]:
from sql_py_defs import *
import matplotlib.pyplot as plt
import numpy as np
import keyring

In [78]:
pw = keyring.get_password("mysql", "ellaguise")
connection = create_db_connection("localhost", "root", pw, "agn_samp")

SQL_password
MySQL Database connection successful


In [27]:
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

In [46]:
q1 = """
SELECT z,lum FROM agn_samp_z_5_area_10;
"""

results = read_query(connection, q1)
z = [results[i][0] for i in range(len(results))]
lum = [results[i][1] for i in range(len(results))]

In [51]:
log_sigma_sq = [8. - 0.27*(lum[i]) + (0.47+np.log10(1+z[i])) for i in range(len(z))]
log_tau = [-8.13 + 0.24*(lum[i]) + (0.34*np.log10(1+z[i])) for i in range(len(z))] 

In [55]:
alter_tau = """
ALTER TABLE agn_samp_z_5_area_10
ADD tau FLOAT;
"""

alter_sigma = """
ALTER TABLE agn_samp_z_5_area_10
ADD sigma_sq FLOAT;
"""

execute_query(connection, alter_tau)
execute_query(connection, alter_sigma)

Query successful
Query successful


In [71]:
sql = '''
    UPDATE agn_samp_z_5_area_10 
    SET tau = %s, sigma_sq = %s
    where id = %s
    '''

    
val = [(float(log_tau[i]), float(log_sigma_sq[i]), int(i)) for i in range(len(log_tau))]


execute_list_query(connection, sql, val)

Query successful
